In [47]:
# verified on 2/16/2025
# format of pass.txt
"""
google:your_email@gmail.com:your_google_app_password
yahoo:your_email@yahoo.com:your_yahoo_app_password
""";

In [48]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import re

# Yahoo IMAP settings
IMAP_SERVER = 'imap.gmail.com'
IMAP_PORT = 993
# EMAIL = 'your_email@gmail.com'
# PASSWORD = 'APP_password'

with open('pass.txt') as fin:
    strs=fin.readlines()
for s in strs:
    if s.split(":")[0]=='google':
        EMAIL=s.split(":")[1].strip()
        PASSWORD=s.split(":")[-1].strip()
# EMAIL,PASSWORD    

# get emails

In [49]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [50]:
# status, validity_data = mail.status(MAILBOX, ['UIDVALIDITY'])
# uid_validity = validity_data[0].split()[2].decode('utf-8').strip(']')
# print(f"Current UIDVALIDITY: {uid_validity}")

In [51]:
start_date = '1-Jan-2024'
end_date = '1-Jul-2024'

search_criteria=f'(SINCE {start_date} BEFORE {end_date})'
# search_criteria=f'(SINCE {start_date})'

# Search for emails within the date range
status, messages = mail.uid('search',None, search_criteria)
# status, messages = mail.search(None, f'SINCE "{start_date}"')
# status, messages = mail.search(None, 'ALL')

# sender_email = 'example@domain.com'
# search_criteria = f'(FROM "{sender_email}" SINCE "{start_date}" BEFORE "{end_date}")'

# Convert the message IDs to a list
target_uids = messages[0].split()

print(f"Found {len(target_uids)} emails.")


Found 2758 emails.


In [52]:
target_uids

[b'51286',
 b'51287',
 b'51288',
 b'51289',
 b'51290',
 b'51291',
 b'51292',
 b'51293',
 b'51294',
 b'51295',
 b'51296',
 b'51297',
 b'51298',
 b'51299',
 b'51300',
 b'51301',
 b'51302',
 b'51303',
 b'51304',
 b'51305',
 b'51306',
 b'51307',
 b'51308',
 b'51309',
 b'51310',
 b'51311',
 b'51312',
 b'51313',
 b'51314',
 b'51315',
 b'51316',
 b'51317',
 b'51318',
 b'51319',
 b'51320',
 b'51321',
 b'51322',
 b'51323',
 b'51324',
 b'51325',
 b'51326',
 b'51327',
 b'51328',
 b'51329',
 b'51330',
 b'51331',
 b'51332',
 b'51333',
 b'51334',
 b'51335',
 b'51336',
 b'51337',
 b'51338',
 b'51339',
 b'51340',
 b'51341',
 b'51342',
 b'51343',
 b'51344',
 b'51345',
 b'51346',
 b'51347',
 b'51348',
 b'51349',
 b'51350',
 b'51351',
 b'51352',
 b'51353',
 b'51354',
 b'51355',
 b'51356',
 b'51357',
 b'51358',
 b'51359',
 b'51360',
 b'51361',
 b'51362',
 b'51363',
 b'51364',
 b'51365',
 b'51366',
 b'51367',
 b'51368',
 b'51369',
 b'51370',
 b'51371',
 b'51372',
 b'51373',
 b'51374',
 b'51375',
 b'51376',

In [ ]:
results=[]
i=0
for uid in target_uids:
    i+=1
    # UID FETCH command
    status, data = mail.uid('fetch', uid, '(RFC822)')
    if status == 'OK' and data != [None]:
        r={}
        # Parse email content
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)
        sender=msg['From']
        print(f"\n{i} UID: {uid}")
        print(f"From: {sender}")
        
        # Decode headers
        subject, encoding = decode_header(msg['Subject'])[0]
        if isinstance(subject, bytes):
            try:
                subject = subject.decode(encoding if encoding else 'utf-8')
            except:
                subject="Decoding Error"        

        print(f"Subject: {subject}")
        
        # Extract the date
        email_date = msg["Date"]
        print(email_date)
        
        r["id"]=uid
        r["sender"]=sender
        r["date"]=email_date
        r["subject"]=subject
        results.append(r)           

    else:
        print(f"Email with UID {uid} not found")


1 UID: b'51286'
From: "Zillow" <instant-updates@mail.zillow.com>
Subject: 3 Results for For Sale
Mon, 01 Jan 2024 08:32:50 +0000

2 UID: b'51287'
From: "Red Lobster" <redlobsteremail@em.redlobster.com>
Subject: Lobster + Shrimp = All. Week. Long.
Mon, 01 Jan 2024 08:15:59 -0500

3 UID: b'51288'
From: "Christopher & Banks" <noreply@em.christopherandbanks.com>
Subject: Happy New Year!🎉
Mon, 01 Jan 2024 07:32:15 -0600

4 UID: b'51289'
From: "Ed (via LinkedIn)" <messages-noreply@linkedin.com>
Subject: I still want to connect
Mon, 1 Jan 2024 15:19:08 +0000 (UTC)

5 UID: b'51290'
From: "Costco Wholesale" <Costco@digital.costco.com>
Subject: Kick-off the New Year with NEW Savings Starting Today!
Mon, 01 Jan 2024 16:25:21 +0000

6 UID: b'51291'
From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
Subject: “senior hardware engineer”: Amazon Web Services (AWS) - Senior Hardware Development Engineer, AWS Server Engineering
Mon, 1 Jan 2024 18:51:20 +0000 (UTC)

7 UID: b'51292'
From: TikTok 

In [ ]:

# Close connection
mail.close()
mail.logout()

# process emails

In [ ]:
def get_email(str_sender):
    pattern = re.compile(r'<(.*?)>')
    match = re.search(pattern, str_sender)

    if match:
        # print("Found:", match.group())
        email=match.group(1)
    else:
        # print("Not found")
        email="NA"   

    sender, encoding = decode_header(str_sender)[0]
    if isinstance(sender, bytes):
        sender = sender.decode(encoding if encoding else 'utf-8')

    return email,sender   

In [ ]:
df=pd.DataFrame(results)
df[["email","sender"]]=df.apply(lambda r:get_email(r['sender']),axis=1,result_type='expand')

In [ ]:
df.head()

In [ ]:
dx=df.groupby("email").count().reset_index().sort_values(by="sender",ascending=True)
dx.tail(20)

In [ ]:
dx.tail(20).email.unique()

In [ ]:
xx=list(dx.email.sort_values(ascending=False).values)
xx

In [ ]:
f1=['zmail@mail.zillow.com',
    'outreach.senate.ca.gov',
    'insideapple.apple.com',
    'medium.com',
    'newsletters.edn.com',
    'noreply@uber.com',
#  'yibingtang@meta.com',
#  'ybtang02@yahoo.com',
 'weekinreview@newsletters.eetimes.com',
 'webinar@newsletters.techonline.com',
 'vtgmail@list.vacationstogo.com',
#  'vimeo@vimeo.com',
 'vacationdeals@blgrnvacations.com',
 'updates@send-p.usnews.com',
#  'updates-noreply@linkedin.com',
#  'uber@uber.com',
 'ttsmarketplace@email.tiktok.com',
 'techpaper@newsletters.techonline.com',
 'survey@merewards.sg',
 'support@sc.mail.deepseek.com',
 'shelbysilva-silvacooptics.com@shared1.ccsend.com',
 'shein@usmail.shein.com',
#  'security@facebookmail.com',
#  'sam.glorit_x@etched.com',
#  'rineyprasanth@gmail.com',
 'redlobsteremail@em.redlobster.com',
#  'pge@email.opower.com',
 'open-houses@mail.zillow.com',
#  'notifications-noreply@linkedin.com',
#  'noreply@uber.com',
#  'noreply@em.pge.com',
#  'no_reply@post.applecard.apple',
#  'no_reply@email.apple.com',
#  'no-reply@em-s.dropbox.com',
#  'no-reply@accounts.google.com',
 'newsletters@e.consumeraffairs.com',
#  'newsletters-noreply@linkedin.com',
 'newsletter@email.kqed.org',
 'newsletter@divenewsletter.com',
 'mouser@info.techonline-direct.com',
#  'messaging-digest-noreply@linkedin.com',
#  'messages-noreply@linkedin.com',
 'members@m.consumeraffairs.com',
 'members@e.consumeraffairs.com',
 'member@email.kqed.org',
 'marketing@thesamragroup.ccsend.com',
 'mail@mail.adobe.com',
#  'mail@eg.hotels.com',
#  'mail@eg.expedia.com',
# #  'magloirevishaal@gmail.com',
#  'linkedin@e.linkedin.com',
#  'jobs-noreply@linkedin.com',
#  'jobs-listings@linkedin.com',
#  'jobalerts-noreply@linkedin.com',
#  'invitations@linkedin.com',
 'instant-updates@mail.zillow.com',
#  'inmail-hit-reply@linkedin.com',
 'info@e.ubm-info.net',
 'iheartradio@listen.iheart.com',
#  'icloudplus@insideapple.apple.com',
#  'hit-reply@linkedin.com',
 'hello@mkt.getyourguide.com',
 'hello@mail.grammarly.com',
#  'groups-noreply@linkedin.com',
 'funfriday@newsletters.edn.com',
 'email@market.temuemail.com',
#  'editors-noreply@linkedin.com',
#  'developer@insideapple.apple.com',
 'delights@enews.merewards.sg',
 'club@castellodiamorosa.com',
 'cameronhart@silvacooptics.ccsend.com',
 'audiexperience@e.audiusa.com',
#  'appstore@insideapple.apple.com',
#  'appletv@insideapple.apple.com',
#  'applepay@insideapple.apple.com',
#  'applenewsplus@insideapple.apple.com',
#  'applemusic@insideapple.apple.com',
#  'applefitnessplus@insideapple.apple.com',
#  'applearcade@insideapple.apple.com',
 'SouthwestAirlines@iluv.southwest.com',
#  'RepRoKhanna@mail.house.gov',
#  'RapidRewards@pts.southwest.com',
#  'PGECustomerService@notifications.pge.com',
 'Nixplay@marketing.nixplay.com',
 'chairman@alamedagop.org',
 'RepRoKhanna@mail.house.gov',
 'updates@focusrite.com',
 'blog.secretprofittools.com',
 'pro.milkymoneyway.com',
 'ldii.longdistanceinvestings.com',
 'first.AmericanTruthCast.com',
 'today.investingwithmind.com',
 'day.usastocksinsider.com',
 "best.marketexpopro.com",
 'net.richpeoplenetworks.com',
 'fin.finacetechfuture.com',
 'editor@nowcc.notesonwealthcreation.com',
 'prisintol.com',
 'tfm.thefinancememories.com',
 'today.investingwithmind.com',
 'ucc.urbancapitalist.com',
 'digest.marketbroadview.com',
 'tips.sportinvestorsleague.com',
 'cottonbubblelay.com',
 'deals@deals.vegas.com',
 'find@via.healthyinvestingtips.com',
 'webinar@newsletters.techonline.com',
 'l.barnett@wtf.wealthtimefuture.com',
 'get@real.multitradersmoves.com',
 'mailing@ceg.certaineconomicgrowth.com',
 'temu@updates.temuofficial.com',
 'strategy@insights.FinancesDataDriven.com',
 'ruffus.white@net.richpeoplenetworks.com',
 'with@top.goldmineadvisors.com',
 'your@very.smartsuccessstrategy.com',
 'news@daily.investmentjourneys.com',
 'check@tips.sportinvestorsleague.com',
 'daily@digest.marketbroadview.com',
 'q.hayes@choice.besttoinvestnews.com',
 'news@daily.financiffy.com',
 'daily@blog.secretprofittools.com',
 'mail@your.usagoldmiracle.com',
 'the.directionforexpo.com',
 'daily@digest.marketbroadview.com',
 'delights@enews.merewards.sg', 
 'Nixplay@marketing.nixplay.com', 
 'member@email.kqed.org',
 'members@e.consumeraffairs.com', 
 'newsletter@divenewsletter.com', 
 'shein@usmail.shein.com',
 ]

In [ ]:
xlist=[]
for x in xx:
    fInlcuded=False
    for y in f1:
        if y in x:
            fInlcuded=True
    if not fInlcuded:
        xlist.append(x) 
print("the following email is not included in the delete list")
xlist

In [ ]:
mustKeepList=['wuxuexiaojie000@gmail.com',
 'updates-noreply@linkedin.com',
 'yibingtang@gmail.com',
 'ybtang02@yahoo.com',
 'hongxiachen@hotmail.com',
 'sam.glorit_x@etched.com',
 'notifications-noreply@linkedin.com',
 'noreply@em.pge.com',
 'no_reply@post-service.applecard.apple',
 'no-reply@em-s.dropbox.com',
 'no-reply@accounts.google.com',
 'messaging-digest-noreply@linkedin.com',
 'messages-noreply@linkedin.com',
 'linkedin@e.linkedin.com',
 'jobs-noreply@linkedin.com',
 'jobs-listings@linkedin.com',
 'jobalerts-noreply@linkedin.com',
 'invitations@linkedin.com',
 'inmail-hit-reply@linkedin.com',
 'hit-reply@linkedin.com',
 'editors-noreply@linkedin.com',
 'appleid@id.apple.com',
 'IDX-noreply@google.com',
 'Costco@digital.costco.com']

In [ ]:
dx[dx.subject>3].email.values

In [ ]:
email_to_delete=f1

In [ ]:
print(len(email_to_delete))
email_to_delete=[x for x in email_to_delete if x not in mustKeepList]
print(len(email_to_delete))

In [ ]:
subject_to_delete =['David Kant','FunWords Team','richard grant','stockpile saving','satelite earnings now','this token could','David Kant','Nextdish','hotels.com', 'Matthew Wehner','Elizabeth Harris', 'Trump','David Kant','Part-Time Work','Investors League','MileagePlus Partner','Russia in Shock','woot.com','Christopher Johnson','Tax-Free Cash Hack']

In [ ]:
pattern1 = '|'.join(subject_to_delete)
pattern2 = '|'.join(email_to_delete)

In [ ]:

dz=df[(df.sender.str.contains(pattern1, case=False)) | (df.email.str.contains(pattern2, case=False))]

In [ ]:
dz

In [ ]:
import time

In [ ]:
testdate=time.strftime("%Y%m%d-%H%M")
print(testdate)
dz[['email','subject','sender','id']].sort_values(by='email').to_csv(f'google_email_delete_record_{testdate}.csv')

In [ ]:
df_remain=pd.merge(df, dz, on=['id','sender','date','email','subject'], how='outer', indicator=True)
df_remain[['email','subject','sender','id']].sort_values(by='email').to_csv(f'google_email_not_delete_record_{testdate}.csv')

In [ ]:
uids_to_delete=list(dz.id.unique())
uids_to_delete, len(uids_to_delete)

# delete emails

In [ ]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [ ]:
# Delete emails
i=0
for uid in uids_to_delete:
    i+=1
    print(i,uid)
    try:
        status, response = mail.uid('store', uid, '+FLAGS', '\\Deleted')
        print(f"Delete status: {status}, Response: {response}")
    except:
        print(uid,"not working")
        
# Expunge deleted emails
mail.expunge()

In [ ]:

# Close connection
mail.close()
mail.logout()